# Google Earth Engine Data Access

This notebook demonstrates using Google Earth Engine (GEE) as an alternative data source for land clearing analysis.

**Advantages of GEE:**
- Global coverage (not limited to Australia)
- Cloud-based processing (no local storage needed)
- Multiple satellite platforms (Landsat, Sentinel-2, MODIS)
- High temporal resolution
- Easy to transfer workflows to other regions

## Setup and Authentication

First time setup:
```bash
# Install GEE libraries
pip install earthengine-api geemap

# Authenticate (opens browser)
earthengine authenticate
```

In [ ]:
# Import GEE data access functions
from aus_land_clearing.data import (
    initialize_gee,
    load_gee_landsat_fc,
    load_gee_sentinel2_fc,
    export_gee_timeseries,
    export_gee_to_drive
)
from aus_land_clearing.utils import get_study_area_bbox, get_time_range
import matplotlib.pyplot as plt

# Initialize GEE
success = initialize_gee()
print(f"GEE initialized: {success}")

## Load Landsat Data from GEE

Using Landsat 8/9 Collection 2 for consistency with DEA timeframe.

In [ ]:
# Define study area (Queensland)
bbox_qld = get_study_area_bbox('queensland')
print(f"Queensland bbox: {bbox_qld}")

# Define time period
time_range = ('2020-01-01', '2023-12-31')
print(f"Time range: {time_range}")

# Load Landsat data
landsat_data = load_gee_landsat_fc(
    bbox=bbox_qld,
    time_range=time_range,
    cloud_cover=30,
    scale=30
)

print(f"\nLoaded {landsat_data['metadata']['count']} Landsat images")
print(f"Source: {landsat_data['metadata']['source']}")

## Load Sentinel-2 Data (Higher Resolution)

Sentinel-2 offers 10m resolution and higher temporal frequency.

In [ ]:
# Load Sentinel-2 data for NSW
bbox_nsw = get_study_area_bbox('new_south_wales')

sentinel_data = load_gee_sentinel2_fc(
    bbox=bbox_nsw,
    time_range=time_range,
    cloud_cover=20,
    scale=10
)

print(f"Loaded {sentinel_data['metadata']['count']} Sentinel-2 images")
print(f"Resolution: {sentinel_data['metadata']['scale']}m")

## Export Time Series Data

Extract NDVI time series for analysis and visualization.

In [ ]:
# Export Landsat NDVI time series
ndvi_ts = export_gee_timeseries(
    landsat_data,
    variable='NDVI',
    output_path='../data/outputs/gee_landsat_ndvi.csv',
    format='csv'
)

print(f"Exported {len(ndvi_ts)} time steps")
print("\nFirst few records:")
print(ndvi_ts.head())

## Simple Visualization

Basic time series plot for quality checking.

In [ ]:
# Simple time series plot
if not ndvi_ts.empty:
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.plot(ndvi_ts['date'], ndvi_ts['NDVI'], marker='o', linestyle='-', linewidth=1)
    ax.set_xlabel('Date')
    ax.set_ylabel('NDVI')
    ax.set_title('NDVI Time Series - Queensland (Google Earth Engine)')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('../data/outputs/gee_ndvi_timeseries.png', dpi=150)
    plt.show()
    
    print("Plot saved to: ../data/outputs/gee_ndvi_timeseries.png")

## Export Large Datasets to Google Drive

For larger exports, use GEE's batch export to Google Drive.

In [ ]:
# Export NDVI composite to Google Drive
export_gee_to_drive(
    landsat_data,
    variable='NDVI',
    description='qld_ndvi_2020_2023',
    folder='AUS_Land_Clearing',
    scale=30
)

print("Export task started. Monitor progress at:")
print("https://code.earthengine.google.com/tasks")

## Comparison: GEE vs DEA

### When to use GEE:
- Need global coverage or comparisons
- Want higher temporal resolution (Sentinel-2: 5-day revisit)
- Prototyping and exploration
- Cloud-based processing (no local infrastructure)
- Transferring workflow to other regions

### When to use DEA:
- Working specifically with Australia
- Need authoritative, policy-relevant products
- Want pre-processed, analysis-ready data
- Require specific Australian products (e.g., fractional cover with PV/NPV/BS)
- Integration with Australian spatial infrastructure

### Best Practice:
- Use **both** sources for different purposes
- GEE for exploration and rapid iteration
- DEA for final authoritative analysis
- Validate GEE results against DEA products

## Next Steps

1. **Process exported data** using time-series analysis functions
2. **Combine GEE and DEA** data for comprehensive analysis
3. **Export clean data** for narrative visualization in other tools
4. **Scale to other regions** using same GEE workflow

See other notebooks for:
- Time-series processing
- Change detection
- Data export for visualization

## Additional Resources

- **GEE Documentation**: https://developers.google.com/earth-engine/
- **geemap Guide**: https://geemap.org/
- **Project References**: See `docs/REFERENCES.md`